чтение и анализ логов бэнчмарка

# Импорты

In [5]:
import re
from pathlib import Path
from pprint import pprint

import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import plotly.express as px
import plotly.graph_objects as go
import seaborn as sns
from plotly import colors
from sklearn.decomposition import PCA
from sklearn.preprocessing import LabelEncoder, MinMaxScaler

from src.utils.read_logs import LogReader

In [6]:
plt.ioff()
%matplotlib notebook

# Чтение логов

In [7]:
log_file_path = "logs/2024-12-05 method_logs.json"
logs = LogReader.load_logs_from_file(log_file_path)

In [8]:
logs[0]

{'method_name': 'TensorLy_Tucker_video-0_pytorch_truncated_svd_svd',
 'method_args': {'rank': [110, 256, 144, 3],
  'n_iter_max': 100,
  'init': 'svd',
  'svd': 'truncated_svd',
  'random_state': 42},
 'qualitative_metrics': {'Language': 'Python',
  'Library': 'TensorLy',
  'TensorLy backend': 'pytorch',
  'Tensor type': 'Dense',
  'Data type': 'video-0',
  'Platform': 'CPU, GPU',
  'Decomposition method': 'Tucker'},
 'quantitative_metrics': {'gpu_allocated_memory_used_mb': [94.03125,
   0.0,
   0.0,
   0.0,
   0.0],
  'gpu_cached_memory_used_mb': [1376.0, 1374.0, 1372.0, 1372.0, 1372.0],
  'ram_mem_used_mb': [2957.69921875,
   2957.703125,
   2957.703125,
   2957.703125,
   2957.71875],
  'duration': [4.0693864822387695,
   4.067977666854858,
   4.028419256210327,
   4.106175184249878,
   4.1296546459198],
  'frobenius_error': [1.9805761176726042,
   1.9805761176726042,
   1.9805761176726042,
   1.9805761176726042,
   1.9805761176726042],
  'compression_ratio': [50.4540892321654,
   5

In [9]:
log_data = []
log_errors = []

for log in logs:
    if log["error_message"] == "":
        try:
            data_entry = {
                # Method name
                "method_name": log["method_name"],
                # Some Qualitative Data
                "decomposition_method": log["qualitative_metrics"]["Decomposition method"],
                "data_type": log["qualitative_metrics"]["Data type"],
                "language": log["qualitative_metrics"]["Language"],
                "library": log["qualitative_metrics"]["Library"],
                "tensor_type": log["qualitative_metrics"]["Tensor type"],
                "platform": log["qualitative_metrics"]["Platform"],
                "backend": log["qualitative_metrics"].get(
                    next(key for key in log["qualitative_metrics"] if "backend" in key)
                ),
                # GPU Allocated Memory
                "gpu_allocated_memory_used_min": np.min(
                    [e for e in log["quantitative_metrics"]["gpu_allocated_memory_used_mb"] if e >= 0.0]
                ),
                "gpu_allocated_memory_used_max": np.max(
                    [e for e in log["quantitative_metrics"]["gpu_allocated_memory_used_mb"] if e >= 0.0]
                ),
                "gpu_allocated_memory_used_mean": np.mean(
                    [e for e in log["quantitative_metrics"]["gpu_allocated_memory_used_mb"] if e >= 0.0]
                ),
                "gpu_allocated_memory_used": np.max(
                    [e for e in log["quantitative_metrics"]["gpu_allocated_memory_used_mb"] if e >= 0.0]
                ),
                # GPU Cached Memory
                "gpu_cached_memory_used_min": np.min(
                    [e for e in log["quantitative_metrics"]["gpu_cached_memory_used_mb"] if e >= 0.0]
                ),
                "gpu_cached_memory_used_max": np.max(
                    [e for e in log["quantitative_metrics"]["gpu_cached_memory_used_mb"] if e >= 0.0]
                ),
                "gpu_cached_memory_used_mean": np.mean(
                    [e for e in log["quantitative_metrics"]["gpu_cached_memory_used_mb"] if e >= 0.0]
                ),
                "gpu_cached_memory_used": np.max(
                    [e for e in log["quantitative_metrics"]["gpu_cached_memory_used_mb"] if e >= 0.0]
                ),
                # RAM Memory Usage
                "ram_mem_used_min": np.min([e for e in log["quantitative_metrics"]["ram_mem_used_mb"] if e >= 0.0]),
                "ram_mem_used_max": np.max([e for e in log["quantitative_metrics"]["ram_mem_used_mb"] if e >= 0.0]),
                "ram_mem_used_mean": np.mean([e for e in log["quantitative_metrics"]["ram_mem_used_mb"] if e >= 0.0]),
                "ram_mem_used": np.max([e for e in log["quantitative_metrics"]["ram_mem_used_mb"] if e >= 0.0]),
                # Duration
                "duration_min": np.min([e for e in log["quantitative_metrics"]["duration"] if e >= 0.0]),
                "duration_max": np.max([e for e in log["quantitative_metrics"]["duration"] if e >= 0.0]),
                "duration_mean": np.mean([e for e in log["quantitative_metrics"]["duration"] if e >= 0.0]),
                "duration": np.max([e for e in log["quantitative_metrics"]["duration"] if e >= 0.0]),
                # Frobenius Error
                "frobenius_error_min": np.min([e for e in log["quantitative_metrics"]["frobenius_error"] if e >= 0.0]),
                "frobenius_error_max": np.max([e for e in log["quantitative_metrics"]["frobenius_error"] if e >= 0.0]),
                "frobenius_error_mean": np.mean(
                    [e for e in log["quantitative_metrics"]["frobenius_error"] if e >= 0.0]
                ),
                "frobenius_error": np.max([e for e in log["quantitative_metrics"]["frobenius_error"] if e >= 0.0]),
                # Compression Ratio
                "compression_ratio_min": np.min(
                    [e for e in log["quantitative_metrics"]["compression_ratio"] if e >= 0.0]
                ),
                "compression_ratio_max": np.max(
                    [e for e in log["quantitative_metrics"]["compression_ratio"] if e >= 0.0]
                ),
                "compression_ratio_mean": np.mean(
                    [e for e in log["quantitative_metrics"]["compression_ratio"] if e >= 0.0]
                ),
                "compression_ratio": np.max([e for e in log["quantitative_metrics"]["compression_ratio"] if e >= 0.0]),
            }
            log_data.append(data_entry)
        except Exception as e:
            print(f"Exception with {log['method_name']}. Error: {e!s}")
    else:
        data_entry = {
            # method name
            "method_name": log["method_name"],
            # qualitative metrics
            "decomposition_method": log["qualitative_metrics"]["Decomposition method"],
            "data_type": log["qualitative_metrics"]["Data type"],
            "language": log["qualitative_metrics"]["Language"],
            "library": log["qualitative_metrics"]["Library"],
            "tensor_type": log["qualitative_metrics"]["Tensor type"],
            "platform": log["qualitative_metrics"]["Platform"],
            "backend": log["qualitative_metrics"].get(
                next(key for key in log["qualitative_metrics"] if "backend" in key)
            ),
            # Error data
            "error_message": log["error_message"],
        }
        log_errors.append(data_entry)

df_logs = pd.DataFrame(log_data)
df_logs_errors = pd.DataFrame(log_errors)

In [10]:
df_logs.shape

(4211, 32)

In [11]:
df_logs_errors.shape

(4142, 9)

## Обработка ошибок

In [12]:
def extract_memory_info(error_message):
    pattern = r"(\d+\.?\d*)\s*(\w+iB)"
    match = re.search(pattern, error_message)

    if match:
        value = float(match.group(1))  # Память
        unit = match.group(2)  # Тип памяти (единица измерения)

        # Конвертация в MiB
        conversion_factors = {
            "KiB": 1 / 1024,  # Килобайты -> Мегабайты
            "MiB": 1,  # Мегабайты -> Мегабайты
            "GiB": 1024,  # Гигабайты -> Мегабайты
            "TiB": 1024**2,  # Терабайты -> Мегабайты
            "PiB": 1024**3,  # Петабайты -> Мегабайты
            "EiB": 1024**4,  # Эксабайты -> Мегабайты
            "ZiB": 1024**5,  # Зеттабайты -> Мегабайты
            "YiB": 1024**6,  # Йоттабайты -> Мегабайты
        }

        if unit in conversion_factors:
            value_in_mib = value * conversion_factors[unit]
        else:
            raise ValueError(f"Неизвестная единица измерения памяти: {unit}")

        return value_in_mib, "MiB"

    return None, None

In [14]:
for index, row in df_logs_errors.iterrows():
    error_message = row["error_message"]

    if "Tried to allocate" in error_message:  # pytorch
        value, unit = extract_memory_info(error_message)
        if not (value and unit):
            print("Не удалось извлечь информацию о памяти из GPU and RAM ошибки.")
    elif "Unable to allocate" in error_message:  # numpy
        value, unit = extract_memory_info(error_message)
        if not (value and unit):
            print("Не удалось извлечь информацию о памяти из RAM ошибки.")
    else:
        print(error_message)
        break

    df_logs_errors.at[index, "memory_tried_to_allocated_error_mib"] = value

In [15]:
df_logs_errors.head()

,method_name,decomposition_method,data_type,language,library,tensor_type,platform,backend,error_message,memory_tried_to_allocated_error_mib
0,TensorLy_Tucker_video-0_pytorch_symeig_svd_svd,Tucker,video-0,Python,TensorLy,Dense,"CPU, GPU",pytorch,"Пропущена итерация из-за ошибки: pytorch, syme...",93306.88
1,TensorLy_Tucker_video-0_numpy_symeig_svd_svd,Tucker,video-0,Python,TensorLy,Dense,"CPU, GPU",numpy,"Пропущена итерация из-за ошибки: numpy, symeig...",93286.40
2,TensorLy_Tucker_video-1_pytorch_symeig_svd_svd,Tucker,video-1,Python,TensorLy,Dense,"CPU, GPU",pytorch,"Пропущена итерация из-за ошибки: pytorch, syme...",52490.24
3,TensorLy_Tucker_video-1_numpy_symeig_svd_svd,Tucker,video-1,Python,TensorLy,Dense,"CPU, GPU",numpy,"Пропущена итерация из-за ошибки: numpy, symeig...",52531.20
4,TensorLy_Tucker_video-2_pytorch_symeig_svd_svd,Tucker,video-2,Python,TensorLy,Dense,"CPU, GPU",pytorch,"Пропущена итерация из-за ошибки: pytorch, syme...",93306.88


# Функции для анализа логов

In [11]:
# Отрисовка горизонтальных бар-графиков для метрик
def plot_barh(ax, x_data, y_data, title, xlabel, best_value=None, best_label=None, color="green"):
    ax.barh(y_data, x_data, color=color)
    ax.set_title(title, fontsize=14)
    ax.set_xlabel(xlabel, fontsize=12)
    if best_value is not None and best_label is not None:
        ax.axvline(x=best_value, color="red", linestyle="--", label=best_label)
        ax.legend()
    for i in range(len(x_data)):
        ax.text(x_data.iloc[i], i, f"{x_data.iloc[i]:.6f}", va="center")

In [12]:
def get_metrics_data(filtered_df):
    filtered_df_unique_by_method_name = filtered_df["method_name"].unique()
    analysis_data = []

    for method in filtered_df_unique_by_method_name:
        method_specific_data = filtered_df[filtered_df["method_name"] == method]

        compression_ratio = method_specific_data["compression_ratio"].to_numpy()[0]
        duration = method_specific_data["duration"].to_numpy()[0]
        total_memory = (
            method_specific_data["ram_mem_used"].to_numpy()[0]
            + method_specific_data["gpu_cached_memory_used"].to_numpy()[0]
        )
        frobenius_error = method_specific_data["frobenius_error"].to_numpy()[0]

        analysis_data.append(
            {
                "method_name": method,
                "duration": duration,
                "total_memory": total_memory,
                "frobenius_error": frobenius_error,
                "compression_ratio": compression_ratio,
            }
        )

    return pd.DataFrame(analysis_data)

In [13]:
# Анализ и визуализация методов по количественным показателям в разрезе качественного показателя
def plot_barhs_and_analyze_dfs(df_logs, group_name: str = "decomposition_method"):
    plt.rcParams.update(
        {
            "axes.titlesize": 16,
            "axes.labelsize": 14,
            "xtick.labelsize": 12,
            "ytick.labelsize": 12,
            "legend.fontsize": 12,
        }
    )
    groups = df_logs[group_name].unique()
    analysis_results = {}

    for group in groups:
        filtered_df = df_logs[df_logs[group_name] == group]

        analysis_df = get_metrics_data(filtered_df)

        best_time_method = analysis_df.loc[analysis_df["duration"].idxmin()]
        best_memory_method = analysis_df.loc[analysis_df["total_memory"].idxmin()]
        best_error_method = analysis_df.loc[analysis_df["frobenius_error"].idxmin()]

        analysis_results[group] = {
            "best_time_method": best_time_method.to_dict(),
            "best_memory_method": best_memory_method.to_dict(),
            "best_error_method": best_error_method.to_dict(),
        }
    #
    #     num_methods = len(analysis_df["method_name"])
    #     height_per_method = 0.5
    #     fig_height = max(36, num_methods * height_per_method)
    #     fig_height = 2 ** 16 - 1 if 2 ** 16 < fig_height else fig_height
    #
    #     fig, axes = plt.subplots(4, 1, figsize=(12, fig_height))
    #
    #     plot_barh(
    #         axes[0],
    #         analysis_df["compression_ratio"],
    #         analysis_df["method_name"],
    #         f"Сравнение сжатия по {group_name} - {group}",
    #         "Сжатие (%)",
    #     )
    #
    #     plot_barh(
    #         axes[1],
    #         analysis_df["duration"],
    #         analysis_df["method_name"],
    #         f"Время исполнения по {group_name} - {group}",
    #         "Время (с)",
    #         best_time_method["duration"],
    #         "Лучший метод",
    #         color="skyblue",
    #     )
    #
    #     plot_barh(
    #         axes[2],
    #         analysis_df["total_memory"],
    #         analysis_df["method_name"],
    #         f"Сумма затрачиваемой памяти по {group_name} - {group}",
    #         "Память (МБ)",
    #         best_memory_method["total_memory"],
    #         "Лучший метод",
    #         color="lightgreen",
    #     )
    #
    #     plot_barh(
    #         axes[3],
    #         analysis_df["frobenius_error"],
    #         analysis_df["method_name"],
    #         f"Ошибка Фробениуса по {group_name} - {group}",
    #         "Ошибка (%)",
    #         best_error_method["frobenius_error"],
    #         "Лучший метод",
    #         color="salmon",
    #     )
    #
    #     plt.tight_layout()
    #     plt.subplots_adjust(top=0.95, bottom=0.05, left=0.05, right=0.95)
    #     plt.show()

    return analysis_results

In [14]:
# # Отрисовка горизонтальных бар-графиков для метрик
# def plot_error_bar(ax, data_mean, data_min, data_max, title, ylabel, label):
#     yerr_lower = data_mean - data_min
#     yerr_upper = data_max - data_mean
#
#     yerr_lower = max(yerr_lower, 0)
#     yerr_upper = max(yerr_upper, 0)
#
#     ax.errorbar(
#         [0], [data_mean], yerr=[[yerr_lower], [yerr_upper]], fmt="o",
#         markersize=10, capsize=10, capthick=3, elinewidth=3, color="lightgreen", label=label
#     )
#     ax.set_title(title)
#     ax.set_ylabel(ylabel)
#     ax.set_xticks([0])
#     ax.set_xticklabels([label])
#
#     for val, _lbl in zip([data_mean, data_min, data_max], ["Mean", "Min", "Max"], strict=False):
#         ax.text(0, val, f"{val:.2f}", ha="center", va="bottom", fontsize=10, fontweight="bold")
#
#     ax.axhline(y=data_mean, color="grey", linestyle="--")
#     ax.axhline(y=data_min, color="lightgrey", linestyle=":")
#     ax.axhline(y=data_max, color="lightgrey", linestyle=":")

In [15]:
# # Анализ и визуализация методов по количественным показателям по минимальным, максимальным и средним значениям
# def plot_error_bars(method_data, method):
#     fig, axes = plt.subplots(3, 2, figsize=(16, 16))
#     fig.suptitle(f"Графики для метода: {method}", fontsize=16, fontweight="bold", color="green")
#
#     plot_error_bar(
#         axes[0, 0],
#         method_data["gpu_allocated_memory_used"].to_numpy()[0],
#         method_data["gpu_allocated_memory_used_min"].to_numpy()[0],
#         method_data["gpu_allocated_memory_used_max"].to_numpy()[0],
#         "GPU Allocated Memory",
#         "Memory (MB)",
#         method,
#     )
#
#     plot_error_bar(
#         axes[0, 1],
#         method_data["gpu_cached_memory_used"].to_numpy()[0],
#         method_data["gpu_cached_memory_used_min"].to_numpy()[0],
#         method_data["gpu_cached_memory_used_max"].to_numpy()[0],
#         "GPU Cached Memory",
#         "Memory (MB)",
#         method,
#     )
#
#     plot_error_bar(
#         axes[1, 0],
#         method_data["ram_mem_used"].to_numpy()[0],
#         method_data["ram_mem_used_min"].to_numpy()[0],
#         method_data["ram_mem_used_max"].to_numpy()[0],
#         "RAM Memory Usage",
#         "Memory (MB)",
#         method,
#     )
#
#     plot_error_bar(
#         axes[1, 1],
#         method_data["duration"].to_numpy()[0],
#         method_data["duration_min"].to_numpy()[0],
#         method_data["duration_max"].to_numpy()[0],
#         "Duration",
#         "Time (s)",
#         method,
#     )
#
#     plot_error_bar(
#         axes[2, 0],
#         method_data["frobenius_error"].to_numpy()[0],
#         method_data["frobenius_error_min"].to_numpy()[0],
#         method_data["frobenius_error_max"].to_numpy()[0],
#         "Frobenius Error",
#         "Error (%)",
#         method,
#     )
#
#     plot_error_bar(
#         axes[2, 1],
#         method_data["compression_ratio"].to_numpy()[0],
#         method_data["compression_ratio_min"].to_numpy()[0],
#         method_data["compression_ratio_max"].to_numpy()[0],
#         "Compression Ratio",
#         "Ratio",
#         method,
#     )
#
#     plt.tight_layout(rect=[0, 0, 1, 0.95])
#     plt.show()

# Расчет лучших метрик

## Сравнение методов по типам данных

In [16]:
analysis_results_data_type = plot_barhs_and_analyze_dfs(df_logs, group_name="data_type")

## Сравнение метрик по методам

In [17]:
analysis_results_decomposition_method = plot_barhs_and_analyze_dfs(df_logs, group_name="decomposition_method")

## Сравнение метрик по методам и типам данных

In [18]:
data_types = df_logs["data_type"].unique()

df_logs_by_data_types_dict = {data_type: df_logs[df_logs["data_type"] == data_type] for data_type in data_types}

analysis_results_data_types_decompositions_methods = {
    data_type: plot_barhs_and_analyze_dfs(df_logs_by_data_types_dict[data_type], group_name="decomposition_method")
    for data_type in data_types
}

# Сравнение метрик

## Метрики в разрезе типов данных

In [19]:
pprint(analysis_results_data_type, indent=4)

{   'eeg-0': {   'best_error_method': {   'compression_ratio': 51.84972227630668,
                                          'duration': 161.12948322296143,
                                          'frobenius_error': 2.628242508687997e-13,
                                          'method_name': 'TensorLy_Tucker_eeg-0_numpy_truncated_svd_random',
                                          'total_memory': 7250.796875},
                 'best_memory_method': {   'compression_ratio': 51.84972227630668,
                                           'duration': 222.25582098960876,
                                           'frobenius_error': 2.9611494187299367e-13,
                                           'method_name': 'TensorLy_Tucker_eeg-0_numpy_truncated_svd_svd',
                                           'total_memory': 7248.21484375},
                 'best_time_method': {   'compression_ratio': 51.84972227630668,
                                         'duration': 10.071362257003784,

## Метрики в разрезе методов

In [20]:
pprint(analysis_results_decomposition_method, indent=4)

{   'CP': {   'best_error_method': {   'compression_ratio': 50.44725543659782,
                                       'duration': 13.776263952255249,
                                       'frobenius_error': 4.157637582053939e-08,
                                       'method_name': 'TensorLy_CP_image-2_pytorch_truncated_svd_random_False_True_abs_rec_error_0_1e-09',
                                       'total_memory': 10885.953125},
              'best_memory_method': {   'compression_ratio': 50.696836830566866,
                                        'duration': 3.1465892791748047,
                                        'frobenius_error': 0.8227858028294301,
                                        'method_name': 'TensorLy_CP_image-1_pytorch_truncated_svd_random_False_True_abs_rec_error_0_1e-08',
                                        'total_memory': 3801.22265625},
              'best_time_method': {   'compression_ratio': 50.696836830566866,
                                     

## Метрики в разрезе типов данных и методов

In [21]:
for data_type, metrics in analysis_results_data_types_decompositions_methods.items():
    print(f"\n{data_type}\n")
    pprint(metrics, indent=4)


video-0

{   'TensorTrain': {   'best_error_method': {   'compression_ratio': 50.34923206676137,
                                                'duration': 0.9326269626617432,
                                                'frobenius_error': 0.3259185481673732,
                                                'method_name': 'TensorLy_TensorTrain_video-0_pytorch_truncated_svd',
                                                'total_memory': 4399.39453125},
                       'best_memory_method': {   'compression_ratio': 50.34923206676137,
                                                 'duration': 11.084824085235596,
                                                 'frobenius_error': 0.32591854816737353,
                                                 'method_name': 'TensorLy_TensorTrain_video-0_numpy_truncated_svd',
                                                 'total_memory': 3956.1640625},
                       'best_time_method': {   'compression_ratio': 50.349232066761

# Аналитика логов количественных и качественных метрик

## Обработка данных

In [22]:
df_logs.shape

(4211, 32)

In [23]:
df_logs.head()

,method_name,decomposition_method,data_type,language,library,tensor_type,platform,backend,gpu_allocated_memory_used_min,gpu_allocated_memory_used_max,...,duration_mean,duration,frobenius_error_min,frobenius_error_max,frobenius_error_mean,frobenius_error,compression_ratio_min,compression_ratio_max,compression_ratio_mean,compression_ratio
0,TensorLy_Tucker_video-0_pytorch_truncated_svd_svd,Tucker,video-0,Python,TensorLy,Dense,"CPU, GPU",pytorch,0.0,94.03125,...,4.080323,4.129655,1.980576,1.980576,1.980576,1.980576,50.454089,50.454089,50.454089,50.454089
1,TensorLy_Tucker_video-0_pytorch_truncated_svd_...,Tucker,video-0,Python,TensorLy,Dense,"CPU, GPU",pytorch,0.0,93.65625,...,2.671328,3.271829,1.980576,1.980576,1.980576,1.980576,50.454089,50.454089,50.454089,50.454089
2,TensorLy_Tucker_video-0_pytorch_symeig_svd_random,Tucker,video-0,Python,TensorLy,Dense,"CPU, GPU",pytorch,0.0,94.03125,...,3.091334,3.148221,1.980576,1.980576,1.980576,1.980576,50.454089,50.454089,50.454089,50.454089
3,TensorLy_Tucker_video-0_pytorch_randomized_svd...,Tucker,video-0,Python,TensorLy,Dense,"CPU, GPU",pytorch,0.0,93.65625,...,8.001925,8.403675,1.980576,1.980576,1.980576,1.980576,50.454089,50.454089,50.454089,50.454089
4,TensorLy_Tucker_video-0_pytorch_randomized_svd...,Tucker,video-0,Python,TensorLy,Dense,"CPU, GPU",pytorch,0.0,94.03125,...,3.088294,3.127742,1.980576,1.980576,1.980576,1.980576,50.454089,50.454089,50.454089,50.454089


In [24]:
df_filtered_for_analytics = df_logs[
    [
        "method_name",
        "data_type",
        "backend",
        "decomposition_method",
        "gpu_allocated_memory_used",
        "gpu_allocated_memory_used_mean",
        "gpu_cached_memory_used",
        "gpu_cached_memory_used_mean",
        "ram_mem_used",
        "ram_mem_used_mean",
        "duration",
        "duration_mean",
        "frobenius_error",
        "frobenius_error_mean",
        "compression_ratio",
        "compression_ratio_mean",
    ]
]

In [25]:
dfs_by_decomposition_method = {
    decomposition_method: df_filtered_for_analytics[
        df_filtered_for_analytics["decomposition_method"] == decomposition_method
    ]
    for decomposition_method in df_filtered_for_analytics["decomposition_method"].unique()
}

In [26]:
for decomposition_method, df_by_decomposition_method in dfs_by_decomposition_method.items():
    enriched_rows = []
    for _, row in df_by_decomposition_method.iterrows():
        matching_log = next((log for log in logs if row["method_name"] == log["method_name"]), None)

        enriched_row = row.copy()

        if row["decomposition_method"] == "Tucker":
            enriched_row["init"] = matching_log["method_args"].get("init")
            enriched_row["svd"] = matching_log["method_args"].get("svd")
        elif row["decomposition_method"] == "TensorTrain":
            enriched_row["svd"] = matching_log["method_args"].get("svd")
        elif row["decomposition_method"] == "CP":
            enriched_row["init"] = matching_log["method_args"].get("init")
            enriched_row["svd"] = matching_log["method_args"].get("svd")
            enriched_row["normalize_factors"] = matching_log["method_args"].get("normalize_factors")
            enriched_row["orthogonalise"] = matching_log["method_args"].get("orthogonalise")
            enriched_row["tol"] = matching_log["method_args"].get("tol")
            enriched_row["l2_reg"] = matching_log["method_args"].get("l2_reg")
            enriched_row["cvg_criterion"] = matching_log["method_args"].get("cvg_criterion")

        enriched_rows.append(enriched_row)

    dfs_by_decomposition_method[decomposition_method] = pd.DataFrame(enriched_rows)

In [27]:
dfs_by_decomposition_method.keys()

dict_keys(['Tucker', 'TensorTrain', 'CP'])

## Обработка ошибок

In [17]:
df_logs_errors.head()

,method_name,decomposition_method,data_type,language,library,tensor_type,platform,backend,error_message,memory_tried_to_allocated_error_mib
0,TensorLy_Tucker_video-0_pytorch_symeig_svd_svd,Tucker,video-0,Python,TensorLy,Dense,"CPU, GPU",pytorch,"Пропущена итерация из-за ошибки: pytorch, syme...",93306.88
1,TensorLy_Tucker_video-0_numpy_symeig_svd_svd,Tucker,video-0,Python,TensorLy,Dense,"CPU, GPU",numpy,"Пропущена итерация из-за ошибки: numpy, symeig...",93286.40
2,TensorLy_Tucker_video-1_pytorch_symeig_svd_svd,Tucker,video-1,Python,TensorLy,Dense,"CPU, GPU",pytorch,"Пропущена итерация из-за ошибки: pytorch, syme...",52490.24
3,TensorLy_Tucker_video-1_numpy_symeig_svd_svd,Tucker,video-1,Python,TensorLy,Dense,"CPU, GPU",numpy,"Пропущена итерация из-за ошибки: numpy, symeig...",52531.20
4,TensorLy_Tucker_video-2_pytorch_symeig_svd_svd,Tucker,video-2,Python,TensorLy,Dense,"CPU, GPU",pytorch,"Пропущена итерация из-за ошибки: pytorch, syme...",93306.88


## Методы для графиков

In [28]:
def preprocess_dataframe(df):
    df_encoded = df.copy()
    for column in df_encoded.columns:
        if df_encoded[column].dtype == "object" and column != "method_name":
            df_encoded[column] = LabelEncoder().fit_transform(df_encoded[column])

    scaler = MinMaxScaler()
    df_encoded[df_encoded.columns.difference(["method_name"])] = scaler.fit_transform(
        df_encoded[df_encoded.columns.difference(["method_name"])]
    )
    return df_encoded

In [29]:
def plot_heatmap(df, title):
    folder_path = Path("../.cache/data_analyze/")

    corr_matrix = df.drop(columns=["method_name"]).corr()

    fig = go.Figure(
        data=go.Heatmap(
            z=corr_matrix.values,
            x=corr_matrix.columns,
            y=corr_matrix.index,
            colorscale=colors.sequential.RdBu,
            colorbar={
                "title": "Correlation",
                "titlefont": {"color": "black"},
                "tickfont": {"color": "black"},
            },
            text=corr_matrix.values,
            texttemplate="%{text:.2f}",
            textfont={"size": 12, "color": "black"},
            hovertemplate="<b>%{x}</b> vs. <b>%{y}</b> <b>Correlation:</b> %{z:.8f}<extra></extra>",
        )
    )

    fig.update_layout(
        title=f"Heatmap: {title}",
        title_font={"color": "black"},
        xaxis={"tickangle": -45, "tickfont": {"size": 12, "color": "black"}},
        yaxis={"tickfont": {"size": 12, "color": "black"}},
        width=1400,
        height=1400,
        plot_bgcolor="white",
        paper_bgcolor="white",
    )

    folder_path.mkdir(parents=True, exist_ok=True)

    fig.write_html(folder_path / f"heatmap_{title}.html")

In [30]:
def plot_pca(df, title):
    folder_path = Path("../.cache/data_analyze/")
    folder_path.mkdir(parents=True, exist_ok=True)

    pca_df = df.drop(columns=["method_name"])
    pca = PCA(n_components=3)
    pca_components = pca.fit_transform(pca_df)

    pca_df = pd.DataFrame(
        {
            "PCA1": pca_components[:, 0],
            "PCA2": pca_components[:, 1],
            "PCA3": pca_components[:, 2],
            "method_name": df["method_name"].to_numpy(),
        }
    )

    fig1 = px.scatter(
        pca_df, x="PCA1", y="PCA2", color="method_name", title=f"PCA 2D Projection: {title}", hover_name="method_name"
    )
    fig1.update_traces(marker={"size": 10})
    fig1.update_layout(
        width=1800,
        height=1400,
        plot_bgcolor="white",
        paper_bgcolor="white",
        title_font={"color": "black"},
        xaxis={
            "titlefont": {"size": 12, "color": "black"},
            "tickfont": {"size": 12, "color": "black"},
        },
        yaxis={
            "titlefont": {"size": 12, "color": "black"},
            "tickfont": {"size": 12, "color": "black"},
        },
        showlegend=False,
    )

    fig1.write_html(folder_path / f"pca_2d_{title}.html")

    fig2 = px.scatter_3d(
        pca_df,
        x="PCA1",
        y="PCA2",
        z="PCA3",
        color="method_name",
        title=f"PCA 3D Projection: {title}",
        hover_name="method_name",
    )
    fig2.update_traces(marker={"size": 5})
    fig2.update_layout(
        scene={
            "xaxis_title": "PCA1",
            "yaxis_title": "PCA2",
            "zaxis_title": "PCA3",
            "xaxis_backgroundcolor": "white",
            "yaxis_backgroundcolor": "white",
            "zaxis_backgroundcolor": "white",
            "xaxis": {
                "titlefont": {"size": 12, "color": "black"},
                "tickfont": {"size": 12, "color": "black"},
            },
            "yaxis": {
                "titlefont": {"size": 12, "color": "black"},
                "tickfont": {"size": 12, "color": "black"},
            },
            "zaxis": {
                "titlefont": {"size": 12, "color": "black"},
                "tickfont": {"size": 12, "color": "black"},
            },
            "bgcolor": "white",
        },
        width=1800,
        height=1400,
        plot_bgcolor="white",
        paper_bgcolor="white",
        title_font={"color": "black"},
        showlegend=False,
    )

    fig2.write_html(folder_path / f"pca_3d_{title}.html")

In [31]:
def plot_pairplot(df, title, sample_size=1000, num_axes=100):
    folder_path = Path("../.cache/data_analyze/")
    folder_path.mkdir(parents=True, exist_ok=True)

    pairplot_df = df.drop(columns=["method_name"])

    if len(pairplot_df) > sample_size:
        pairplot_df = pairplot_df.sample(n=sample_size, random_state=42)

    selected_axes = pairplot_df.columns[:num_axes]

    sns.set(style="white")
    g = sns.pairplot(pairplot_df[selected_axes], diag_kind="kde", markers="+", plot_kws={"s": 50})
    g.fig.suptitle(f"Pairplot: {title}", y=1.02, fontsize=16, color="black")

    for ax in g.axes.flatten():
        ax.tick_params(axis="x", labelsize=12, labelcolor="black")
        ax.tick_params(axis="y", labelsize=12, labelcolor="black")
        ax.set_facecolor("white")
        ax.set_xticks(ax.get_xticks())
        ax.set_xticklabels(ax.get_xticklabels(), rotation=45, ha="right")

    plt.savefig(folder_path / f"pairplot_{title}.png", dpi=300, bbox_inches="tight")
    plt.close("all")

In [32]:
dfs_by_decomposition_method_encoded = {}

In [33]:
dfs_by_decomposition_method.keys()

dict_keys(['Tucker', 'TensorTrain', 'CP'])

In [34]:
for key, df_by_decomposition_method in dfs_by_decomposition_method.items():
    dfs_by_decomposition_method_encoded[key] = preprocess_dataframe(
        df_by_decomposition_method.drop("decomposition_method", axis=1)
    )

In [35]:
# dfs_by_decomposition_method['CP']

In [36]:
# dfs_by_decomposition_method_encoded['CP']

In [37]:
dfs_by_decomposition_method_encoded.keys()

dict_keys(['Tucker', 'TensorTrain', 'CP'])

## Heatmap

In [38]:
method_name = "Tucker"
plot_heatmap(dfs_by_decomposition_method_encoded[method_name], title=method_name)

In [39]:
method_name = "TensorTrain"
plot_heatmap(dfs_by_decomposition_method_encoded[method_name], title=method_name)

In [40]:
method_name = "CP"
plot_heatmap(dfs_by_decomposition_method_encoded[method_name], title=method_name)

## PCA

In [41]:
method_name = "Tucker"
plot_pca(dfs_by_decomposition_method_encoded[method_name], title=method_name)

In [42]:
method_name = "TensorTrain"
plot_pca(dfs_by_decomposition_method_encoded[method_name], title=method_name)

In [43]:
method_name = "CP"
plot_pca(dfs_by_decomposition_method_encoded[method_name], title=method_name)

## Pairplot

In [44]:
method_name = "Tucker"
plot_pairplot(dfs_by_decomposition_method_encoded[method_name], title=method_name)

<IPython.core.display.Javascript object>

In [45]:
method_name = "TensorTrain"
plot_pairplot(dfs_by_decomposition_method_encoded[method_name], title=method_name)

<IPython.core.display.Javascript object>

In [46]:
method_name = "CP"
plot_pairplot(dfs_by_decomposition_method_encoded[method_name], title=method_name)

<IPython.core.display.Javascript object>

# Аналитика логов ошибок